# Setup

In [1]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/pednet_design/code


In [2]:
from mobilkit.umni import *
from setup import P, City, Pednet

In [3]:
from igraph import Graph
from pqdm.processes import pqdm

In [4]:
aus = City('Austin, TX')
cam = City('Cambridge, MA')
tor = City('City of Toronto, Ontario', 'Toronto')

In [5]:
cities = [cam, tor, aus]

---
# Create pednet variants

In [6]:
od_sample = 's1'

In [7]:
%%time
# t=0:25    
pns = [Pednet(*params, od_sample) for params in tqdm([
    # Cambridge
    (cam, 'exists', 'current'),
    (cam, 'exists or is_xwalk', 'crosswalks'),
    # Austin
    (aus, 'exists', 'current'),
    (aus, 'exists or ~is_xwalk', 'sidewalks'),
    (aus, 'exists or is_xwalk', 'crosswalks'),
    (aus, None, 'both'),
    # Toronto
    (tor, 'exists', 'current'),
    (tor, 'exists or ~is_xwalk', 'sidewalks'),
    (tor, 'exists or is_xwalk', 'crosswalks'),
    (tor, None, 'both'),
])]
for c in cities:
    c.pn = {pn.name: pn for pn in pns if pn.city == c}

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 23.3 s, sys: 3.48 s, total: 26.7 s
Wall time: 25.8 s


## Get edge betweenness

In [8]:
def get_edge_bwness(pn):
    pn.E['bwness'] = pn.G.edge_betweenness(False, weights=pn.E['len'])
    pn.E.to_parquet(U.kmfile(pn.city.root + f'edges/{pn.name}.parquet'))
    
# %time x = get_edge_bwness(pns[4]); x

In [9]:
# t~=8:10:00 {cambridge=>0:02, austin=>4:51:00+..., toronto=>1:31:00+...}
# %time for c in tqdm([cam, aus, tor]): print(c); get_edge_bwness(c)

---
# Get shortest paths

## Define shortest path functions for `pqdm`

### New (per pednet)

In [10]:
func_template = 'def sp_{}_{}(o, d): return pns[{}].get_sp(o, d)'
for i, pn in enumerate(pns):
    exec(func_template.format(pn.city.name, pn.name, i))

## Compute shortest paths

In [25]:
def get_shortest_path(pn, n_jobs=30, save=True, overwrite=False):
    outfile = pn.city.root / f'paths/{pn.name}.parquet'
    if outfile.exists() and not overwrite:
        return pd.read_parquet(outfile)
    # filter unique connected OD pairs
    od = (pn.od.groupby(['vid_o','vid_d'])['n_ods'].sum().reset_index()
          .merge(pn.V['cid'].rename('cid_o'), left_on='vid_o', right_index=True)
          .merge(pn.V['cid'].rename('cid_d'), left_on='vid_d', right_index=True)
          .query('cid_o == cid_d').drop(columns=['cid_o','cid_d']))
    func = globals()[f'sp_{pn.city.name}_{pn.name}']
    sp = pqdm(zip(od['vid_o'], od['vid_d']), func, n_jobs=n_jobs,
              total=od.shape[0], argument_type='args')
    try:
        sp = (od.merge(Pdf(sp), on=('vid_o','vid_d'))
              .sort_values(['vid_o','vid_d']).reset_index(drop=True))
        if save:
            sp.to_parquet(U.mkfile(outfile))
    except Exception as e:
        print(e)
        return sp
    return sp

# %time x = get_shortest_path(cam.pn['current']); x

In [31]:
%%time
# t=4:59:00 {A0=>28:09, A1=>1:08:00, A2=>1:19:00, A3=>2:25:00, C0=>2:59, C1=>3:09, T0=>9:07, T1=>15:43, T2=>9:37, T3=>16:24}
for pn in tqdm(pns):
    pn.sp = get_shortest_path(pn, n_jobs=60)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4.15 s, sys: 2.43 s, total: 6.57 s
Wall time: 5.78 s


## Compute edge weight (volume) & pednet score

In [38]:
def get_pednet_score(pn, save=True, overwrite=False):
    fpath = pn.city.root / f'edges/{pn.name}.parquet'
    E = gpd.read_parquet(fpath)
    if 'pedscore' in E.columns and not overwrite:
        return E
    E = E.drop(columns='n_trips', errors='ignore')
    sp = (pn.sp.set_index('n_ods')['edges'].rename('id').explode().reset_index()
          .groupby('id')['n_ods'].sum().rename('n_trips').reset_index())
    E = E.merge(sp, on='id', how='left').fillna({'n_trips': 0})
    E['pedscore'] = np.log10(1 + pn.E['n_trips'] * pn.E['bwness'])
    pn.E = E.astype({'n_trips': int})
    if save:
        pd.E.to_parquet(fpath)
    
# %time x = get_pednet_score(cam.pn['current']); x

In [39]:
%time for pn in tqdm(pns): get_pednet_score(pn) # t = 1min 30s

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 4.59 s, sys: 1.75 s, total: 6.34 s
Wall time: 5.02 s
